# Model Evaluation

## Imoprt libraries

In [250]:
import numpy as np
import pandas as pd
from dataclasses import dataclass, field
from enum import Enum
from typing import List
import os

## Settings

In [53]:
SETTING_RESULT_PARENT_DIRECTORY_PATH = os.path.join("Result")
SETTING_TEST_DATA_FRAME_PATH = os.path.join("test_data_frame.csv")
SETTING_REVENUE_TEST_DATA_FRAME_PATH = os.path.join(
    os.pardir, "datasets", "revenue_test_data_by_date_store.csv"
)
SETTING_STATES = [
    "CA_1",
    "CA_2",
    "CA_3",
    "CA_4",
    "TX_1",
    "TX_2",
    "TX_3",
    "WI_1",
    "WI_2",
    "WI_3",
]
SETTING_FIRST_FORECAST_DAY = 1914

## Enums

In [252]:
class Metric(Enum):
    MASE = 0  # Mean Absolute Scaled Error
    RMSSE = 1  # Root Mean Squared Scaled Error

## Classes

In [253]:
@dataclass
class Data_Frame:
    name: str
    data_frame: pd.DataFrame


@dataclass
class MetricResult:
    data_frame_name: str
    metric: Metric
    value: float


@dataclass
class Result:
    author: str
    data_frame_list: List[Data_Frame] = field(default_factory=list)
    metric_result_list: List[MetricResult] = field(default_factory=list)

## Metric formulas

### Mean Absolute Scaled Error

In [254]:
def mase(train: np.ndarray, actual: np.ndarray, forecast: np.ndarray) -> float:
    """
    Compute Mean Absolute Scaled Error (MASE) based on the correct formula.

    Args:
        train (np.ndarray): Array of historical (training) data
        actual (np.ndarray): Array of actual values for the test period
        forecast (np.ndarray): Array of forecasted values

    Returns:
        float: MASE value
    """

    # Compute scale denominator
    denominator = np.mean(np.abs(np.diff(train)))

    if denominator == 0:
        return np.nan  # Avoid division by zero

    # Compute numerator
    numerator = np.mean(np.abs(actual - forecast))

    return np.sqrt(numerator / denominator)

### Root Mean Squared Scaled Error

In [255]:
def rmsse(train: np.ndarray, actual: np.ndarray, forecast: np.ndarray) -> float:
    """
    Compute Root Mean Squared Scaled Error (RMSSE) based on the correct formula.

    Parameters:
        train (np.ndarray): Array of historical (training) data
        actual (np.ndarray): Array of actual values for the test period
        forecast (np.ndarray): Array of forecasted values

    Returns:
        float: RMSSE value
    """

    # Compute scale denominator
    denominator = np.mean(np.square(np.diff(train)))

    if denominator == 0:
        return np.nan  # Avoid division by zero

    # Compute numerator
    numerator = np.mean(np.square(actual - forecast))

    return np.sqrt(numerator / denominator)

## Helper Functions

### Create test data frame

In [58]:
def create_test_data_frame() -> pd.DataFrame:
    if os.path.isfile(SETTING_TEST_DATA_FRAME_PATH):
        print(f"Test data frame already exists at {SETTING_TEST_DATA_FRAME_PATH}")
        test_df = pd.read_csv(SETTING_TEST_DATA_FRAME_PATH)
        return test_df.copy()

    if not os.path.isfile(SETTING_REVENUE_TEST_DATA_FRAME_PATH):
        raise FileNotFoundError(
            f"Revenue test data frame not found at {SETTING_REVENUE_TEST_DATA_FRAME_PATH}."
        )

    revenue_test_df = pd.read_csv(SETTING_REVENUE_TEST_DATA_FRAME_PATH)

    pivoted_revenue_test_df = revenue_test_df.pivot(
        index=["date", "d", "weekday"], columns="store_id", values="revenue"
    ).reset_index()

    pivoted_revenue_test_df.insert(0, "id", [(i + 1) for i in range(1941)])

    pivoted_revenue_test_df.to_csv(SETTING_TEST_DATA_FRAME_PATH, index=False)

    return pivoted_revenue_test_df.copy()

### Read results

In [257]:
def read_result(parent_directory: str) -> List[Result]:
    # Check if the given path exists and is a directory
    if not os.path.isdir(parent_directory):
        raise NotADirectoryError(f"Directory '{parent_directory}' is not found.")

    # Get a list of all subdirectories within the parent directory
    member_subdirectories = [
        d
        for d in os.listdir(parent_directory)
        if os.path.isdir(os.path.join(parent_directory, d))
    ]

    # Check if there are any subdirectories; if not, raise an error
    if not member_subdirectories:
        raise NotADirectoryError("Author subdirectories are not found.")

    result_list: List[Result] = []

    for member in member_subdirectories:
        result = Result(author=member, data_frame_list=[], metric_result_list=[])

        member_path = os.path.join(parent_directory, member)

        csv_files = [
            f
            for f in os.listdir(member_path)
            if f.endswith(".csv") and os.path.isfile(os.path.join(member_path, f))
        ]

        for csv_file in csv_files:
            data_frame = Data_Frame(name=csv_file, data_frame=pd.read_csv(csv_file))
            result.data_frame_list.append(data_frame)

        result_list.append(result)

    return result_list

### Evaluate results

In [60]:
def evaluate(
    evaluated_data_frame: pd.DataFrame, test_data_frame: pd.DataFrame, metric: Metric
) -> float:
    train = test_data_frame[test_data_frame["id"] < SETTING_FIRST_FORECAST_DAY]
    actual = test_data_frame[test_data_frame["id"] >= SETTING_FIRST_FORECAST_DAY]

    match metric:
        case Metric.RMSSE:
            value = np.sum(
                np.array(
                    [
                        rmsse(
                            train=train[state].to_numpy(),
                            actual=actual[state].to_numpy(),
                            forecast=evaluated_data_frame[state].to_numpy(),
                        )
                        for state in SETTING_STATES
                    ]
                )
            )

        case Metric.MASE:
            value = np.sum(
                np.array(
                    [
                        mase(
                            train=train[state].to_numpy(),
                            actual=actual[state].to_numpy(),
                            forecast=evaluated_data_frame[state].to_numpy(),
                        )
                        for state in SETTING_STATES
                    ]
                )
            )

    return value


def evaluate_result(
    result_list: List[Result], test_data_frame: pd.DataFrame, metric: Metric
):
    for result in result_list:
        result.metric_result_list.extend(
            [
                MetricResult(
                    data_frame_name=data_frame.name,
                    metric=metric,
                    value=evaluate(data_frame.data_frame, test_data_frame, metric),
                )
                for data_frame in result.data_frame_list
            ]
        )

## Main Flow

### Create test data frame

In [259]:
test_df = create_test_data_frame()

Test data frame already exists at test_data_frame.csv


In [260]:
test_df

,id,date,d,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
0,1,2011-01-29,d_1,10933.16,9101.52,11679.83,4561.59,6586.68,9915.78,7597.99,6454.72,5451.46,9367.88
1,2,2011-01-30,d_2,9787.06,8417.53,12161.46,4681.41,6610.60,9804.54,7356.54,5645.77,4636.86,9868.80
2,3,2011-01-31,d_3,7201.38,5320.51,9123.86,3637.98,4551.97,6651.16,5406.70,3640.12,4621.58,7551.65
3,4,2011-02-01,d_4,7407.74,5550.56,10249.78,3708.92,5374.39,6985.60,5597.97,2949.96,5754.75,7181.53
4,5,2011-02-02,d_5,6566.12,5229.72,9538.65,3841.14,4347.07,6039.05,4069.74,2.96,2679.19,4646.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936,1937,2016-05-18,d_1937,12920.62,12766.25,17780.83,8116.41,9851.97,11121.03,12078.07,9605.89,12928.75,9163.29
1937,1938,2016-05-19,d_1938,13259.90,13432.94,18635.35,8367.57,8403.09,10474.33,11196.76,10478.86,13547.39,9660.13
1938,1939,2016-05-20,d_1939,13999.65,15545.28,18219.23,8960.17,11296.88,13832.01,14667.00,11358.75,14139.33,11982.37
1939,1940,2016-05-21,d_1940,18637.70,24088.59,23849.52,9768.51,13137.35,15212.81,15696.64,14614.05,15020.25,12370.23


### Read results

In [261]:
result_list = read_result(SETTING_RESULT_PARENT_DIRECTORY_PATH)

In [262]:
if not result_list:
    raise FileNotFoundError("No data frame found.")

### Evaluate models

#### Metric: Mean Absolute Scaled Error

In [263]:
evaluate_result(result_list=result_list, test_data_frame=test_df, metric=Metric.MASE)

#### Metric: Root Mean Squared Scaled Error

In [264]:
evaluate_result(result_list=result_list, test_data_frame=test_df, metric=Metric.RMSSE)